# Astrofísica Computacional
## Proyecto Final: Problema Gravitacional de los N-Cuerpos

### Desarrollado por: 
### Joar Esteban Buitrago Carrillo
### Juan Sebastian Montoya Combita  

  

###Observatorio Astronómico Nacional  
2021

Las ecuaciones de movimiento de N-Partículas moviendose bajo su interacción gravitacional mutua se puede escribir en la forma.
$$m_{i}\ddot{x}_{i} = -Gm_{i} \sum_{j=1, i \neq j}^{N} \frac{m_{j}}{|x_{ij}|^{2}}$$
Donde $x_{ij}=x_{i}-x_{j}$ es el vector que apunta desde la partícula $j$ a la partícula $i$.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.animation as Animation
#from IPython.display import HTML #these last 2 are necesarry to play the animation
#from base64 import b64encode
from astropy import constants as const
from astropy import units as u
import plotly.graph_objects as go

##   1.   Generalización:
Implemente un código que resuelva el problema diferencial de las ecuaciones de movimiento utilizando un método Runge-Kutta de orden 4. El algoritmo debe ser lo suficientemente general para poder incluír un número arbitrario de partículas y las condiciones deben ser leídas de un archivo.






1.   Primero, se crea la clase principal para cada cuerpo



In [5]:
class Astrobject:
  def __init__(
      self,
      pos = np.zeros(3) * u.au, 
      vel = np.zeros(3) * u.au/ u.year,
      mass=1. * u.M_sun
  ):
    self.pos= pos
    self.vel = vel
    assert (mass > 0) 
    self.mass = mass
    self.dpos, self.dvel = _initzeros() 

  def _initzeros():
    return ([np.zeros(3)* u.au, np.zeros(3) * u.au/ u.year])

  def acc(self, mass, distance): #ODE 
    up = const.G * mass * distance
    down = (np.linalg.norm(distance)) ** 3
    return (-up/down)

  def add_interaction(self, other): #Rudge Kuta 4 Implementation
    orbit_mass = other.mass 
    distance = (self.pos - other.pos).to(u.au)
    k1 = self.vel * dt
    vk1 = self.acc(orbit_mass, distance) * dt
    k2 = (self.vel + (vk1/2)) * dt
    vk2 = self.acc(orbit_mass, distance + (k1/2)) * dt
    k3 = (self.vel + (vk2/2)) * dt
    vk3 = self.acc(orbit_mass, distance + (k2/2)) * dt
    k4 = (self.vel + vk3) * dt
    vk4 = self.acc(orbit_mass, distance + k3) * dt
    self.dpos += (k1+ 2*k2 + 2*k3 + k4) / 6
    self.dvel += (vk1 + 2*vk2 + 2*vk3 + vk4) / 6

  def step(self):
    self.pos += self.dpos
    self.vel += self.dvel
    self.dpos, self.dvel = _initzeros()


2.   Creación de la función que guarda un arreglo de clases para cada cuerpo



In [4]:
def nbodies(bd): #bd debe tener las unidades asignadas según el dato importado
  bodies = np.array(np.shape(bd)[0]) #Crea el arreglo vacío en donde en cada posición se insertara cada cuerpo representado en una clase
  bodies = np.array(Astrobject(bd[i,0:3].to(u.au),bd[i,3:6].to(u.au/ au.year), bd[6].to(u.M_sun)) for i in bd) #Crea la clase para cada objeto con las unidades UA Years SolarMass
  return (bodies)



3.   Función del main loop



In [6]:
def astroloop(bodies, time):
  N = np.shape(bodies)[0] 
  bodiespos= np.zeros((N, time, 3))
  bodiesvel= np.zeros((N, time, 3))
  for t in range(time):
    for j in range(N):
      for k in range(N):
        if (j != k):
          bodies[j].add_iteraction(bodies[k])
    for l in range(N):
      bodies[l].step()
      bodiespos[t,l] = bodies[l].pos
      bodiesvel[t,l] = bodies[l].vel
  return (bodiespos, bodiesvel)



4.   Función de graficación



In [8]:
def astroplotter(bodiespos, colors, labels): #Previamente debe definir el arreglo de colores y etiquetas
  fig = go.Figure()
  for t in range(np.shape(bodiespos)[0]):
    fig.add_trace(go.Scatter3d(
        x = bodiespos[t,:,0],
        y = bodiespos[t,:,1],
        z = bodiespos[t,:,2],
        size = 2,
        color= colors[t],
        name = labels[t]))
  fig.show()

## 2.   Simulación sistema Sol-Tierra:
Con el fin de probar el código implementado, utilice los datos iniciales para el sistema Sol-Tierra dados en el archivo sun_earth.dat y grafique el comportamiento del sistema a lo largo de algunos años. Compruebe que la orbita no se comporta a la forma de una espiral.



## 3.   Aseguramiento de Convergencia:
Con el fin de asegurar la convergencia del algoritmo, implemente una rutina dentro del programa que calcule la energía total del sistema de N-partículas. Evalue el comportamiento de la energía a lo largo de la evolución y grafique para comprobar si existe algún cambio significativo. ¿El comportamiento mejora o empeora al modificar el paso de la integración?



## 4.   Simulación Sistema estrellas alrededor de SgrA*:
Ahora utilice su código para estudiar la evolución del sistema de 13 estrellas $SO$ moviendose alrededor del agujero negro supermasivo $SgrA*$, proporcionado en el archivo SOstars.dat. Verifique también el comportamiento de la energía del sistema de 13+1 partículas. Grafique las orbitas de estas estrellas a lo largo de un periodo de 100 años.


